In [37]:
import pandas as pd
import numpy as np
import datetime as dt

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [29]:
#Cargo los archivos
eventos = pd.read_csv("../events_up_to_01062018.csv")
labels = pd.read_csv("../labels_training_set.csv")

C:\Users\felip\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
#Veo las columnas de eventos 
list(eventos)

['timestamp',
 'event',
 'person',
 'url',
 'sku',
 'model',
 'condition',
 'storage',
 'color',
 'skus',
 'search_term',
 'staticpage',
 'campaign_source',
 'search_engine',
 'channel',
 'new_vs_returning',
 'city',
 'region',
 'country',
 'device_type',
 'screen_resolution',
 'operating_system_version',
 'browser_version']

In [77]:
#Dropeo columnas que son irrelevantes
eventos.drop(columns=['skus', 'channel','search_term','campaign_source','search_engine','screen_resolution','operating_system_version','browser_version'])

,timestamp,event,person,url,sku,model,condition,storage,color,staticpage,new_vs_returning,city,region,country,device_type,fecha,mes,anio,year,month


In [78]:
#Me quedo con los set de datos que coincidan con los labels y sean inferiores al 01/06/2018
set_entrenamiento = pd.merge(eventos, labels, on='person', how='inner')


In [81]:
# Split-out validation dataset
array = eventos.values
X = array[:,0:4]
Y = array[:,4]
validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)